# ADX

* Trend indicator
* Shows that there is a trend and what it its strength
* With more indicators the ADX is useful to indicate when to get in or out of an operation 



* Derived from other two indicators
  * **+DMI** *(Positive Directional Movement Index)* 
  * **-DMI** *(Negative Directional Movement Index)* 
* Both are **oscillators**, varying between $0$ and $100$ and using an moving average with a window of $14$ days

 

* The ADX results in a number between $0$ and $100$
* The indicator does not show if it's a trend where the prices will increase or decrease, only its **strength**
* Usually 
  * $ADX < 20$ - *no trend*
  * $ADX > 20 \ and \ ADX < 40 $ - *there is a trend*
  * $ADX > 40 $ - *there is a strong trend*

* Accordingly to investopedia



* | **ADX** **Value** | **Trend Strength**     |
  | ----------------- | ---------------------- |
  | 0-25              | Absent or Weak Trend   |
  | 25-50             | Strong Trend           |
  | 50-75             | Very Strong Trend      |
  | 75-100            | Extremely Strong Trend |



## Why?

* There is a problem with **moving average** based indicators - Take the *HiLo* as an example
  * If the market becomes **lateral** *i.e* no trend - with a narrow channel in the Bollinger Bands with the price quickly varying: a lot of signals will be fired just to be canceled a few minutes later   



**So** we could **combine** the *ADX* with the *HiLo* and only fire the *HiLo* orders when the *ADX* indicates a strong trend



## Do it - five steps to the stars

1. Identify the **True Range** (*TR*)

   1. `a = today.high - today.min`
   2. `b = abs(today.high - yesterday.close)`
   3. `c = abs(today.low - yesterday.close)`
   4. `TR = max(a, b, c)`

2. Calculate the **Directional Movement** (*+-DM*)

   1. *+DM*

      * ```pseudocode
        if (today.high - yesterday.high) > (yesterday.low - today.low) then:
        	plus_dm = today.high - yesterday.high
        
        else:
        	plus_dm = 0
        
        ```

   2. *-DM*

      * ```pseudocode
        if (yesterday.low - today.low) > (today.high - yesterday.high) then:
        	minus_dm = yesterday.low - today.low
        
        else:
        	minus_dm = 0
        ```

3. Smooth *TR*, *+DM* and *-DM*

   1. Apply a *SMA* with a *window of*  **14** *days*

4. Calculate *+DMI* and *-DMI*

   1. $DMI_+ = \frac{smooth(DM_+, window)}{smooth(TR, window)} * 100$
   2. $DMI_- = \frac{smooth(DM_-, window)}{smooth(TR, window)} * 100$

5. Calculate *ADX*

   1. $DX = abs(DMI_+ - DMI_-)$
   2. $ADX = smooth(DX, window)$

   

In [45]:
import sys
import os

import chart_studio.plotly as plty
import plotly.graph_objs as gobjs

import plotly

sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets


account = ReadSecrets()\
        .set_secrets_path('../secrets.json')\
        .set_default_accessors()\
        .access('plotly')['main_account']

plty.sign_in(account['username'], account['apiKey'])


from datetime import datetime as calendar
import numpy as np

from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
yf.pdr_override()

from DataHelper import DataHelper

from plotly import tools

In [46]:
data, close, extractors = DataHelper.get_history_formatted(['AAPL', 'MSFT', '^GSPC', 'VALE3.SA'], calendar(2018, 1 ,3), calendar(2020, 2, 1))
msft = extractors['MSFT']()
vale3 = extractors['VALE3.SA']()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [47]:
vale3_prev = vale3.shift(1)

# intermediate dataset, whose purpose is to find out the TR value
tr = pd.DataFrame([
        vale3.High - vale3.Low, 
        np.absolute(vale3.High - vale3_prev.Close), 
        np.absolute(vale3.Low - vale3_prev.Close)])

# to facilitate our calculation, we have to transpose the matrix, 
tr = tr.transpose()


# get the max value of each row
tr = tr.max(axis=1)

# assign TR to our dataset
vale3['TR'] = tr

display(vale3)

,Open,High,Low,Close,Adj Close,Volume,TR
Date,,,,,,,
2018-01-03,41.830002,41.880001,41.299999,41.470001,38.826576,12744200,0.580002
2018-01-04,41.810001,42.369999,41.520000,41.639999,38.985741,18433000,0.899998
2018-01-05,41.570000,42.290001,41.310001,42.290001,39.594303,15251300,0.980000
2018-01-08,42.400002,43.230000,42.400002,43.230000,40.474384,14542800,0.939999
2018-01-09,43.580002,43.750000,42.930000,43.070000,40.324589,15986200,0.820000
...,...,...,...,...,...,...,...
2020-01-27,51.660000,51.820000,50.480000,50.509998,50.509998,38779700,3.320000
2020-01-28,51.200001,52.090000,51.080002,51.200001,51.200001,23046400,1.580002
2020-01-29,51.700001,51.889999,50.459999,50.750000,50.750000,23323000,1.430000


In [48]:
# get +DM
dm_plus = np.where((vale3.High - vale3_prev.High) > (vale3_prev.Low - vale3.Low), 
                     (vale3.High - vale3_prev.High).apply(
                             lambda x: np.max([x, 0])),
                     0)

# get -DM
dm_minus = np.where((vale3_prev.Low - vale3.Low) > (vale3.High - vale3_prev.High), 
                     (vale3_prev.Low - vale3.Low).apply(
                             lambda x: np.max([x, 0])),
                     0)


vale3['plus_DM'] = dm_plus
vale3['minus_DM'] = dm_minus

display(vale3.head())
display(vale3.tail())

,Open,High,Low,Close,Adj Close,Volume,TR,plus_DM,minus_DM
Date,,,,,,,,,
2018-01-03,41.830002,41.880001,41.299999,41.470001,38.826576,12744200,0.580002,0.000000,0.000000
2018-01-04,41.810001,42.369999,41.520000,41.639999,38.985741,18433000,0.899998,0.489998,0.000000
2018-01-05,41.570000,42.290001,41.310001,42.290001,39.594303,15251300,0.980000,0.000000,0.209999
2018-01-08,42.400002,43.230000,42.400002,43.230000,40.474384,14542800,0.939999,0.939999,0.000000
2018-01-09,43.580002,43.750000,42.930000,43.070000,40.324589,15986200,0.820000,0.520000,0.000000


,Open,High,Low,Close,Adj Close,Volume,TR,plus_DM,minus_DM
Date,,,,,,,,,
2020-01-27,51.660000,51.820000,50.480000,50.509998,50.509998,38779700,3.320000,0.00,3.209999
2020-01-28,51.200001,52.090000,51.080002,51.200001,51.200001,23046400,1.580002,0.27,0.000000
2020-01-29,51.700001,51.889999,50.459999,50.750000,50.750000,23323000,1.430000,0.00,0.620003
2020-01-30,49.970001,51.500000,49.799999,51.500000,51.500000,23083200,1.700001,0.00,0.660000
2020-01-31,50.959999,51.060001,49.799999,50.270000,50.270000,29988200,1.700001,0.00,0.000000


In [49]:
window = 14

vale3['TR_smooth'] = np.NaN
vale3['plus_DM_smooth'] = np.NaN
vale3['minus_DM_smooth'] = np.NaN

# Smooth process using window elements
vale3['TR_smooth'].at[vale3.index[window-1]] = np.sum(vale3.iloc[0:window].TR)
vale3['plus_DM_smooth'].at[vale3.index[window-1]] = np.sum(vale3.iloc[0:window]['plus_DM'])
vale3['minus_DM_smooth'].at[vale3.index[window-1]] = np.sum(vale3.iloc[0:window]['minus_DM'])


def smooth(df, column_result, column_from, current_index, last_index, window):
    return df[column_result][last_index] - (df[column_result][last_index] / window) + df[column_from][current_index] 



for index_num, (index_date, row) in enumerate(vale3[vale3.index[window]:].iterrows()):
    index_num = index_num + window
    
    last_i = vale3.index[index_num - 1]
    '''
    vale3['TR_smooth'].at[index_date]       = vale3.TR_smooth[last_i]        -  (vale3.TR_smooth[last_i] / window)         +  vale3.TR[index_date]
    vale3['plus_DM_smooth'].at[index_date]  = vale3.plus_DM_smooth[last_i]   -  (vale3.plus_DM_smooth[last_i] / window)    +  vale3.plus_DM[index_date]
    vale3['minus_DM_smooth'].at[index_date] = vale3.minus_DM_smooth[last_i]  -  (vale3.minus_DM_smooth[last_i] / window)   +  vale3.minus_DM[index_date]
    '''
    
    vale3['TR_smooth'].at[index_date]       = smooth(vale3, 'TR_smooth', 'TR', index_date, last_i, window)
    vale3['plus_DM_smooth'].at[index_date]  = smooth(vale3, 'plus_DM_smooth', 'plus_DM', index_date, last_i, window)
    vale3['minus_DM_smooth'].at[index_date] = smooth(vale3, 'minus_DM_smooth', 'minus_DM', index_date, last_i, window)

vale3 = vale3.dropna()

display(vale3.head())

,Open,High,Low,Close,Adj Close,Volume,TR,plus_DM,minus_DM,TR_smooth,plus_DM_smooth,minus_DM_smooth
Date,,,,,,,,,,,,
2018-01-22,43.099998,43.160000,42.220001,42.650002,39.931362,11580900,0.939999,0.00,0.379997,11.820004,3.189999,2.150002
2018-01-23,41.990002,42.110001,40.810001,40.930000,38.320992,22075600,1.840000,0.00,1.410000,12.815718,2.962142,3.406430
2018-01-24,41.209999,42.130001,41.139999,41.779999,39.116810,22096600,1.200001,0.02,0.000000,13.100310,2.770560,3.163113
2018-01-25,41.779999,41.779999,41.779999,41.779999,39.116810,0,0.000000,0.00,0.000000,12.164574,2.572663,2.937177
2018-01-26,41.849998,41.880001,40.709999,41.430000,38.789124,19259500,1.170002,0.00,1.070000,12.465677,2.388902,3.797378


In [57]:
vale3['plus_DMI'] = (vale3.plus_DM_smooth / vale3.TR_smooth) * 100
vale3['minus_DMI'] = (vale3.minus_DM_smooth / vale3.TR_smooth) * 100

vale3.head()

,Open,High,Low,Close,Adj Close,Volume,TR,plus_DM,minus_DM,TR_smooth,plus_DM_smooth,minus_DM_smooth,plus_DMI,minus_DMI
Date,,,,,,,,,,,,,,
2018-01-22,43.099998,43.160000,42.220001,42.650002,39.931362,11580900,0.939999,26.988136,18.189517,11.820004,3.189999,2.150002,26.988136,18.189517
2018-01-23,41.990002,42.110001,40.810001,40.930000,38.320992,22075600,1.840000,23.113349,26.580094,12.815718,2.962142,3.406430,23.113349,26.580094
2018-01-24,41.209999,42.130001,41.139999,41.779999,39.116810,22096600,1.200001,21.148816,24.145332,13.100310,2.770560,3.163113,21.148816,24.145332
2018-01-25,41.779999,41.779999,41.779999,41.779999,39.116810,0,0.000000,21.148816,24.145332,12.164574,2.572663,2.937177,21.148816,24.145332
2018-01-26,41.849998,41.880001,40.709999,41.430000,38.789124,19259500,1.170002,19.163833,30.462669,12.465677,2.388902,3.797378,19.163833,30.462669


In [58]:
# get DX value
vale3['DX'] = (np.absolute(vale3.plus_DMI - vale3.minus_DMI) / (vale3.plus_DMI + vale3.minus_DMI)) * 100

vale3.head()

,Open,High,Low,Close,Adj Close,Volume,TR,plus_DM,minus_DM,TR_smooth,plus_DM_smooth,minus_DM_smooth,plus_DMI,minus_DMI,DX
Date,,,,,,,,,,,,,,,
2018-01-22,43.099998,43.160000,42.220001,42.650002,39.931362,11580900,0.939999,26.988136,18.189517,11.820004,3.189999,2.150002,26.988136,18.189517,19.475601
2018-01-23,41.990002,42.110001,40.810001,40.930000,38.320992,22075600,1.840000,23.113349,26.580094,12.815718,2.962142,3.406430,23.113349,26.580094,6.976262
2018-01-24,41.209999,42.130001,41.139999,41.779999,39.116810,22096600,1.200001,21.148816,24.145332,13.100310,2.770560,3.163113,21.148816,24.145332,6.615681
2018-01-25,41.779999,41.779999,41.779999,41.779999,39.116810,0,0.000000,21.148816,24.145332,12.164574,2.572663,2.937177,21.148816,24.145332,6.615681
2018-01-26,41.849998,41.880001,40.709999,41.430000,38.789124,19259500,1.170002,19.163833,30.462669,12.465677,2.388902,3.797378,19.163833,30.462669,22.767746


In [62]:
vale3['ADX'] = np.NaN

# average of 14 periods 
vale3['ADX'].at[vale3.index[window-1]] = vale3.DX[0:window].mean()

# get data from 14th row until end of the dataset
for index_num, (index_date, row) in enumerate(vale3[vale3.index[window]:].iterrows()):
    index_num = index_num + window
    last_i = vale3.index[index_num - 1]
    
    vale3.ADX.at[index_date] = (vale3.ADX[last_i] * (window - 1) + vale3.DX[index_date]) / window
    
vale3 = vale3.dropna()
vale3.head()

,Open,High,Low,Close,Adj Close,Volume,TR,plus_DM,minus_DM,TR_smooth,plus_DM_smooth,minus_DM_smooth,plus_DMI,minus_DMI,DX,ADX
Date,,,,,,,,,,,,,,,,
2018-02-08,41.689999,42.130001,41.070000,41.590000,38.938923,18928800,1.060001,16.241833,22.996570,15.328205,2.489582,3.524961,16.241833,22.996570,17.214606,12.148480
2018-02-09,41.400002,42.400002,41.029999,42.000000,39.322792,22760500,1.370003,16.546172,20.977427,15.603336,2.581755,3.273178,16.546172,20.977427,11.809250,12.124249
2018-02-15,45.009998,45.959999,44.950001,45.950001,43.021004,31043100,3.959999,32.291193,16.474666,18.448811,5.957341,3.039380,32.291193,16.474666,32.433607,13.574917
2018-02-16,45.820000,46.189999,45.470001,46.029999,43.095905,21107300,0.719997,32.277210,15.810182,17.851036,5.761816,2.822281,32.277210,15.810182,34.243959,15.051277
2018-02-19,46.099998,46.799999,45.959999,46.450001,43.489132,12756100,0.840000,34.222965,15.047632,17.415963,5.960259,2.620690,34.222965,15.047632,38.918411,16.756073


In [ ]:


trace_candles = gobjs.Candlestick(x=vale3.index,
                    name="VALE3.SA",
                    open=vale3.Open,
                    high=vale3.High,
                    low=vale3.Low,
                    close=vale3.Close)

trace_di_minus = gobjs.Scatter(
                    x=vale3.index,
                    y=vale3.minus_DMI,
                    name = "-DI",
                    line = dict(color = '#B22222'),
                    opacity = 1)

trace_di_plus = gobjs.Scatter(
                    x=vale3.index,
                    y=vale3.plus_DMI,
                    name = "+DI",
                    line = dict(color = '#17BECF'),
                    opacity = 1)

trace_adx = gobjs.Scatter(
                    x=vale3.index,
                    y=vale3.ADX,
                    name = "ADX",
                    line = dict(color = '#9932CC'),
                    opacity = 1)

fig = dict(data=[trace_di_minus, trace_di_plus, trace_adx])

plty.iplot(fig, filename='adx-line')



In [79]:
fig = plotly.subplots.make_subplots(rows=2, cols=1)

fig.append_trace(trace_candles, 1, 1)
fig.append_trace(trace_di_minus, 2, 1)
fig.append_trace(trace_di_plus, 2, 1)
fig.append_trace(trace_adx, 2, 1)


fig['layout']['yaxis2'].update(range=[0, 100])

plty.iplot(fig, filename='adx-line')